https://www.booking.com

In [1]:
from selenium import webdriver
from fake_useragent import UserAgent
import time
from datetime import datetime, timedelta

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import logging
from urllib3.exceptions import NewConnectionError
from selenium.webdriver.support import expected_conditions as EC



In [2]:
def get_start_end_date():
    start_date = datetime.today()
    end_date = start_date + timedelta(days=60)
    
    return start_date, end_date

In [3]:
ua = UserAgent()
user_agent = ua.random

options = webdriver.FirefoxOptions()
options.add_argument(f'user-agent={user_agent}')
options.add_argument('--headless')
driver = webdriver.Firefox(options=options)

start_date, end_date = get_start_end_date()

#récupère le reservations d'hotels sur une durée de 1 jour
while start_date < end_date:
    
    end_trip = start_date + timedelta(days=1)
    start_date_str = str(start_date).split(' ')
    end_trip_str = str(end_trip).split(' ')
    
    print("new date")
    
    try:
        driver.get(f"https://www.booking.com/searchresults.fr.html?ss=Paris&ssne=Paris&ssne_untouched=Paris&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIARXYAQHoAQGIAgGoAgO4Avjj3akGwAIB0gIkYzY5NTVhYzItYjQwYS00YWJlLWIyMWItOTc0OTAxNmVmYmNj2AIF4AIB&sid=ddc37917e9355cbdcad54b3c452b053a&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1456928&dest_type=city&checkin="+start_date_str[0]+"&checkout="+end_trip_str[0]+"&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure")
                      
    except NewConnectionError:
        logging.error(f"Erreur de connexion pour la date "+start_date+" et la date "+end_trip)
        try:
            driver.get(f"https://www.booking.com/searchresults.fr.html?ss=Paris&ssne=Paris&ssne_untouched=Paris&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARfIARXYAQHoAQGIAgGoAgO4Avjj3akGwAIB0gIkYzY5NTVhYzItYjQwYS00YWJlLWIyMWItOTc0OTAxNmVmYmNj2AIF4AIB&sid=ddc37917e9355cbdcad54b3c452b053a&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1456928&dest_type=city&checkin="+start_date_str[0]+"&checkout="+end_trip_str[0]+"&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure")

        except Exception as e:
            logging.error(f"impossible de scrapper cette date : "+start_date+" et "+end_trip)
            driver.quit()
            continue
    time.sleep(2)
    
    # def extract_data(element, xpath):
    #     try:
    #         element = driver.find_element(By.XPATH, xpath).get_attribute('textContent')
    #     except:
    #         element = ""
    #     return element
    
    link_elements = driver.find_elements(By.CLASS_NAME, 'a78ca197d0')
    print(len(link_elements))
    url_list = []
    
    for link_element in link_elements:
        url = link_element.get_attribute("href")
        if url is not None:
            url_list.append(url)
    
    for url in url_list:
        try:
            driver.get(url)
        except Exception as e:
            logging.error("impossible de se connecter")
            logging.error("reessaie...")
            try:
                driver.get(url)
            except Exception as e:
                logging.error("pas de connexion frero")
                continue
            
            
        try:
            nom_chambre = driver.find_element(By.CSS_SELECTOR, '.d2fee87262').get_attribute('textContent')
        except Exception as e:
            logging.error("we y a pas de nom?")
        try :
            prix_initiale = driver.find_element(By.CSS_SELECTOR, 'td.hp-price-left-align:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)').get_attribute('textContent')
            print(prix_initiale)
        except Exception as e:
            logging.error("no reduction")
        try :
            taux_reduction = driver.find_element(By.CSS_SELECTOR, 'td.hp-price-left-align:nth-child(3) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1) > span:nth-child(1) > span:nth-child(1)').get_attribute('textContent')
            print(taux_reduction) 
        except Exception as e:
            logging.error("no reduction")    
        try :
            prix_apres_reduction = driver.find_element(By.CSS_SELECTOR, 'td.hp-price-left-align:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)').get_attribute('textContent')
        except Exception as e:
            logging.error("comment ca y a pas de prix?")
        # prix_initiale = extract_data(hotel, '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[3]/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/span/div/span')
        # prix_apres_reduction = extract_data(hotel, '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[3]/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/span/div/div/span[1]')
        # nom_chambre = extract_data(hotel, '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[3]/div[1]/div[2]/div/div[1]/div/div[1]/div/div[1]/div/h3/a/div[1]')
        # taux_reduction = extract_data(hotel, '//*[@id="b2searchresultsPage"]')
        print(nom_chambre)
        print(prix_apres_reduction)
        time.sleep(2)

start_date = end_trip

driver.quit()

new date
26

€ 954

-40 %
Paris j'Adore Hotel & Spa

€ 576


€ 954

-40 %
Paris j'Adore Hotel & Spa

€ 576


€ 954

-40 %
Paris j'Adore Hotel & Spa

€ 576



ERROR:root:we y a pas de nom?
ERROR:root:no reduction
ERROR:root:no reduction
ERROR:root:comment ca y a pas de prix?


Paris j'Adore Hotel & Spa

€ 576



ERROR:root:no reduction
ERROR:root:no reduction


Appartement luxe Avenue Foch /Arc de Triomphe ,champs Elysées air conditionne 4P

€ 684



...

Cosy apt Arc de Triomphe

€ 185



ERROR:root:we y a pas de nom?
ERROR:root:no reduction
ERROR:root:no reduction
ERROR:root:comment ca y a pas de prix?


Cosy apt Arc de Triomphe

€ 185



ERROR:root:we y a pas de nom?
ERROR:root:no reduction
ERROR:root:no reduction
ERROR:root:comment ca y a pas de prix?


Cosy apt Arc de Triomphe

€ 185

